In [24]:
import numpy as np
import jax.numpy as jnp
from jax.scipy.special import logsumexp
import jax
from numpy.random.mtrand import noncentral_chisquare
from torch._C import NoneType
from tqdm.notebook import tqdm
import _pickle as cPickle
import pickle
import copy
from jax import jit, vmap, pmap, grad, value_and_grad
import random
from torchvision.datasets import MNIST
from torchvision.datasets import KMNIST
from torchvision.datasets import CIFAR100
from torchvision.datasets import EMNIST
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from jax.example_libraries import stax, optimizers
import torchvision
import torch
from numpy.random import default_rng
from sklearn.neighbors import NearestNeighbors
import torch.utils.data as data_utils
from jax.flatten_util import ravel_pytree
import os
import time
import shutil
import _pickle as cPickle
from math import e
import time
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, LogSoftmax
from sklearn.model_selection import train_test_split
from jax import random
import matplotlib.pyplot as plt
from torchvision import transforms
from logging import Logger
import time
import itertools

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
known_npy = ['anvil',
 'bathtub',
 'bicycle',
 'baseball bat',
 'book',
 'arm',
 'asparagus',
 'airplane',
 'blackberry',
 'bee',
 'blueberry',
 'barn',
 'apple',
 'banana',
 'bench',
 'bear',
 'bandage',
 'The Great Wall of China',
 'basketball',
 'basket']

unknown_npy = ['beach',
 'aircraft carrier',
 'boomerang',
 'axe',
 'beard',
 'bat',
 'bird',
 'ambulance',
 'animal migration',
 'The Mona Lisa',
 'binoculars',
 'bed',
 'birthday cake',
 'angel',
 'alarm clock',
 'belt',
 'baseball',
 'The Eiffel Tower',
 'ant',
 'backpack']


if not any(".npy" in item for item in known_npy):
  known_npy = [x+".npy" for x in known_npy]

if not any(".npy" in item for item in unknown_npy):
  unknown_npy = [x+".npy" for x in unknown_npy]


In [27]:
'''Load data from npy files and normalize and standardize
train_npy and test_npy set manually'''



data_image_size = 28

def normalize_standardize(arr):
  arr = np.array(arr)/255
  arr = (arr-arr.mean())/arr.std()
  return arr


x_train_known_dataset = []
x_test_known_dataset = []
y_train_known_dataset = []
y_test_known_dataset = []

x_train_unknown_dataset = []
x_test_unknown_dataset = []
y_train_unknown_dataset = []
y_test_unknown_dataset = []

known_list = []
unknown_list = []

data_image_size = 28
data_image_depth = 1
number_datapoints_npy = 5000
#data_filepath = "./Quickdraw_data/"
data_filepath = "/content/drive/MyDrive/Colab Notebooks/Handdraw/data/"



if not any(".npy" in item for item in known_npy):
  known_npy = [x+".npy" for x in known_npy]

for idx, files in enumerate(os.listdir(data_filepath)):
  if "npy" in files:
    file_path = data_filepath+files
    npy_load = np.load(file_path, encoding = 'latin1', allow_pickle = True)
    rng = default_rng(idx)
    idx_list = rng.choice(len(npy_load), size = number_datapoints_npy*2, replace = False)
    idx_list_train = idx_list[number_datapoints_npy:]
    idx_list_test = idx_list[:number_datapoints_npy]


    if files in known_npy:

      x_train_known_dataset.extend(npy_load[idx_list_train])
      x_test_known_dataset.extend(npy_load[idx_list_test])
      y_train_known_dataset.extend([idx]*number_datapoints_npy)
      y_test_known_dataset.extend([idx]*number_datapoints_npy)
      known_list.append(idx)

    if files in unknown_npy:
      x_train_unknown_dataset.extend(npy_load[idx_list_train])
      x_test_unknown_dataset.extend(npy_load[idx_list_test])
      y_train_unknown_dataset.extend([idx]*number_datapoints_npy)
      y_test_unknown_dataset.extend([idx]*number_datapoints_npy)
      unknown_list.append(idx)

x_train_known_dataset = normalize_standardize(x_train_known_dataset).reshape(-1,data_image_size,data_image_size)
y_train_known_dataset = np.array(y_train_known_dataset)
x_test_known_dataset = normalize_standardize(x_test_known_dataset).reshape(-1,data_image_size,data_image_size)
y_test_known_dataset = np.array(y_test_known_dataset)


y_train_unknown_dataset = np.array(y_train_unknown_dataset)
x_train_unknown_dataset = normalize_standardize(x_train_unknown_dataset).reshape(-1,data_image_size,data_image_size)
y_test_unknown_dataset = np.array(y_test_unknown_dataset)
x_test_unknown_dataset = normalize_standardize(x_test_unknown_dataset).reshape(-1,data_image_size,data_image_size)



## **Funktions**

Changes in Jax Jit function (sometimes?) need a cache restart for changes to be applied. Happened with def accuracy

In [28]:
def pathandstuff():

    global save_txt
    global base_path
    global save_path

    if os.path.exists(googledrive_path):
        print("on google")
        base_path = googledrive_path
    else:
        raise ValueError('Please create folder for save path or connect to Google Drive')
        
    logs_path = base_path
    '''Set logging and temp paths'''
    timestamp = time.strftime("%d.%m.%Y_%H.%M")
    foldername = timestamp
    save_path = os.path.join(logs_path,foldername,)
    save_path = save_path+"/"
    save_txt = os.path.join(save_path, 'Log_Jax_MNist_{}.txt'.format(foldername))
      
    if use_pickle:
      save_path = pickle_path.split("best_weight_")[0] 
      for i in os.listdir(save_path):
        if "Log" in i:
          save_txt = os.path.join(save_path, i)

    print("Log path:",save_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    return save_path

'''Save current notebook'''
def logg_script(file_name, save_path):
  source = f"/content/drive/MyDrive/Colab Notebooks/{file_name}.ipynb"
  destination = save_path+f"{file_name}.ipynb"
  shutil.copy2(source, destination)

'''logging to txt and print'''
def logg(string_, array = None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    print(string_)
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    print(string_, array)

def log_variables():
    try:
    
      logg(("n_training_epochs = {}".format(n_training_epochs)))
      logg(("n_offsp_epoch = {}".format(n_offsp_epoch)))
      logg(("n_samples = {}".format(n_samples)))
      logg(("n_test = {}".format(n_test)))
      logg(("batch_size = {}".format(batch_size)))
      logg(("std_modifier = {}".format(std_modifier)))
      logg(("use_pickle = {}".format(use_pickle)))
      logg(("pickle_path = {}".format(pickle_path)))
      logg(("NNin1 = {}".format(NNin1)))
      logg(("NNout1 = {}".format(NNout1)))
      logg(("Convu_in1 = {}".format(Convu1_in)))
      logg(("Convu2_in = {}".format(Convu2_in)))
      logg(("kernelsize_ = {}".format(kernelsize_)))
      logg(("n_metaepochs = {}".format(n_metaepochs)))   
      logg(("n_offsp_epoch = {}".format(n_offsp_epoch)))
      logg(("n_offsprings = {}".format(n_offsprings)))
    except Exception:
      pass

In [29]:
def logg_GPU():
  try:
    affe=!nvidia-smi
    string=""
    for i in affe:
      string+=i
    logg("GPU:",string.split("=========||   0  ")[1].split("            Off  | 000")[0])
  except:
    pass

In [30]:
'''creating offsprings, filling treeleaf of 0 and 1 with gaussian noise, doesnt seem to be a problem, ex in offspring_list[0][5]'''


def jax_create_offsprings(key,n_offspr,  fath_weights,std_var):
  statedic_list = []
  randomizer_list = []
  for i in range(0,n_offspr):
    std_modifier = std_var
    rng = jax.random.PRNGKey(key+i)
    random_value_tree = tree_random_normal_like(rng,fath_weights,std_modifier)
    son = jax.tree_map(lambda x,y: x+y, fath_weights,random_value_tree)
    statedic_list.append(son)
    return statedic_list, []

def random_split_like_tree(rng_key, target = None, treedef = None):
    if treedef is None:
        treedef = jax.tree_util.tree_structure(target)
    keys = jax.random.split(rng_key, treedef.num_leaves)
    return jax.tree_util.tree_unflatten(treedef, keys)


def tree_random_normal_like(rng_key, target,std_modifier):
    keys_tree = random_split_like_tree(rng_key, target)
    return jax.tree_map(
        lambda l, k: jax.random.normal(k, l.shape, l.dtype)*std_modifier,
        target,
        keys_tree,
    )

In [31]:
'''softmax for offspring list
    checked 11.04 working correctly'''
def softmax_offlist(off_list,acc_list,temp):
  softmax_list = softmax_result(acc_list,temp)
  for i in range(len(off_list)):
    if i ==  0:
      top_dog = jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
    else:
      general_dog = jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
      top_dog = jax.tree_map(lambda x,y: x+y, top_dog,general_dog)
  return top_dog


'''Creates softmax/temp list out of accuracy list [0.2,0.3,....,0.8]'''
def softmax_result(results,temp: float):
    x = [z/temp for z in results]
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [32]:
def sigma_decay(start, end, n_iter):
  return(end/start)**(1/n_iter)

In [33]:
def cat_dataloader(x,y,random_categories, random_state, n_samples, n_test, number_different_catruns):
  x_train = np.empty((n_samples*number_different_catruns,data_image_size,data_image_size))
  x_test = np.empty((n_test*number_different_catruns,data_image_size,data_image_size))
  y_train = np.empty((n_samples*number_different_catruns,))
  y_test = np.empty((n_test*number_different_catruns,))

  for i in range(number_different_catruns):

    '''Choose 5 random categories from data'''
    x_data0 = x[np.isin(y, random_categories[i]).flatten()]
    y_data0 = y[np.isin(y, random_categories[i]).flatten()]

    '''relabel categories to 0-5 to avoid accuracy bug in mlp'''
    map_dic = dict(zip(random_categories[i], list(range(number_training_categories))))
    y_data0 = np.vectorize(map_dic.get)(y_data0)

    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(x_data0, y_data0, train_size = n_samples,
                                                  test_size = n_test,stratify = y_data0,
                                                  random_state = random_state)

    x_train[i*n_samples:(i+1)*n_samples] = x_train_temp
    x_test[i*n_test:(i+1)*n_test] = x_test_temp
    y_train[i*n_samples:(i+1)*n_samples] = y_train_temp
    y_test[i*n_test:(i+1)*n_test] = y_test_temp

  '''Cast to Jax Array'''
  x_train = jnp.array(x_train,dtype = "float32").reshape(len(x_train), -1)
  x_test = jnp.array(x_test,dtype = "float32").reshape(len(x_test), -1)
  y_train = jnp.array(y_train)
  y_test = jnp.array(y_test)

  '''Reshape for Jax Vectorization'''
  x_train = x_train.reshape(-1,int((n_samples/batch_size)),batch_size,data_image_size,data_image_size)
  y_train = y_train.reshape(-1,int((n_samples/batch_size)),batch_size)
  x_test = x_test.reshape(-1,n_test,data_image_size,data_image_size)
  y_test = y_test.reshape(-1,n_test)

  return x_train, x_test, y_train, y_test

In [34]:
def init_MLP(layer_widths, parent_key, scale = 0.01):

    params = []
    keys = jax.random.split(parent_key, num = len(layer_widths)-1)

    for in_width, out_width, key in zip(layer_widths[:-1], layer_widths[1:], keys):
        weight_key, bias_key = jax.random.split(key)
        params.append([scale*jax.random.normal(weight_key, shape = (out_width, in_width)),
                       scale*jax.random.normal(bias_key, shape = (out_width,))
                       ])
    return params



In [35]:
@jit
def MLP_predict(params, x):

    hidden_layers = params[:-1]
    activation = x

    for w, b in hidden_layers:
        activation = jax.nn.relu(jnp.dot(w, activation) + b)

    w_last, b_last = params[-1]
    logits = jnp.dot(w_last, activation) + b_last

    return logits - logsumexp(logits)

jit_MLP_predict = jit(MLP_predict)

@jit
def batched_MLP_predict(params,x):
  return vmap(jit_MLP_predict, (None, 0))(params,x)
  
jit_batched_MLP_predict = jit(batched_MLP_predict)


In [36]:
Convu1_in = 32
Convu2_in = 64
kernelsize_ = (3,3)

conv_init, conv_apply = stax.serial(
    stax.Conv(Convu1_in,kernelsize_, padding = "SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu2_in, kernelsize_, padding = "SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Flatten,
    stax.Dense(50),
)


In [37]:
def random_cat_list(categories, number_different_catruns, number_training_categories):
    combs = np.array(list(itertools.combinations(categories, number_training_categories)))
    sample = np.random.randint(len(combs), size = number_different_catruns)
    affe = np.array(combs[sample])
    return affe

In [38]:
def load_best_weight_queue(pickle_path):
  count = 0
  best_weight_queue = []
  base_bath = pickle_path.split("best_weight_")[0]
  for idx,i in enumerate(os.listdir(base_bath)[::-1]):
    if "best_weight" in i and count < len_best_weight_queue:
      
      with open(base_bath+i, "rb") as input_file:
        best_weight_queue.append(cPickle.load(input_file))
      count = count+1
  print(f"Top {count} weights imported from Drive")
  return best_weight_queue

In [39]:
@jit
def loss_fn(params, imgs, gt_lbls):
  
    predictions = jit_batched_MLP_predict(params, imgs)
    #print("predictions",predictions.shape)
    return -jnp.mean(predictions * gt_lbls)
    
jit_loss_fn = jit(loss_fn)


@jit
def update(step, params, imgs, gt_lbls, opt_state,lr = 0.01):
    loss, grads = value_and_grad(loss_fn)(params, imgs, gt_lbls)
    opt_state = opt_update(step, grads, opt_state)

    return loss, jax.tree_map(lambda p, g: p - lr*g, params, grads), opt_state

'''@jit
def update(params, imgs, gt_lbls,lr = 0.01):
    loss, grads = value_and_grad(loss_fn)(params, imgs, gt_lbls)

    return loss, jax.tree_map(lambda p, g: p - lr*g, params, grads)'''

jit_update = jit(update)

@jit
def accuracy(conv_weights,MLP_params, dataset_imgs, dataset_lbls):

    imgs = conv_apply(conv_weights, dataset_imgs.reshape(-1,data_image_size,data_image_size,data_image_depth))
    pred_classes = jnp.argmax(jit_batched_MLP_predict(MLP_params, imgs.reshape(-1,NNin1)), axis = 1)

    return jnp.mean(dataset_lbls ==   pred_classes+jnp.min(dataset_lbls))
    
jit_accuracy = jit(accuracy)


In [40]:
'''For loop is neccesary to do batch training. Every update iteration needs to run with updated MPL params'''
@jit
def train(conv_weights, imgs, lbls,MLP_params,opt_state ):



  for n in range(n_training_epochs):  
    for i in range(jnp.shape(imgs)[0]):

      gt_labels = jax.nn.one_hot(lbls[i], number_training_categories)
      img_conv = conv_apply(conv_weights, imgs[i].reshape(-1,data_image_size,data_image_size,data_image_depth))
      #loss, MLP_params = jit_update(MLP_params, img_conv.reshape(-1,NNin1), gt_labels)
      step = (n+i)
      loss, MLP_params,opt_state = jit_update(step, MLP_params, img_conv.reshape(-1,NNin1), gt_labels,opt_state)
  return MLP_params
  
jit_train = jit(train)  

In [41]:
@jit
def bootstrapp_offspring_MLP(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  

  MLP_params_trained = jit_train(conv_weights, batch_affe, labelaffe,
                               #MLP_params,
                               params,opt_state)
 
  
  result = jit_accuracy(conv_weights,MLP_params_trained,test_images,test_lbls)
  return result,MLP_params_trained

jit_bootstrapp_offspring_MLP = jit(bootstrapp_offspring_MLP)  

@jit
def vmap_bootstrapp_offspring_MLP(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  return vmap(jit_bootstrapp_offspring_MLP, ( None, None,None, 0,0,0,0))(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls)
  
jit_vmap_bootstrapp_offspring_MLP = jit(vmap_bootstrapp_offspring_MLP)


In [42]:
def get_varlist(results):
  var_list = []
  for i in results:
    means_of_every_offspring = np.mean(i,axis = 2)
    '''average standard deviation of the mean of every offspring, over all oepochs. How accurate is the fitness evaluation of one offspring?'''
    var = np.var(means_of_every_offspring, axis = 0)
    var_list.append(var)
  return var_list

In [43]:
def get_gen_mean_std (var_list):
  offspringmeans_overallruns = []
  offspringsts_overallruns = []
  for offspring_number in range(np.shape(var_list)[1]):
    singleoff_varlist = []
    for run_number in range(np.shape(var_list)[0]):
      singleoff_varlist.append(var_list[run_number][offspring_number])
    
    mean, std = get_mean_and_std(singleoff_varlist)
    offspringmeans_overallruns.append(mean)
    offspringsts_overallruns.append(std)

  total_generation_mean = np.mean(offspringmeans_overallruns)
  total_generation_average_std = np.mean(np.array(offspringsts_overallruns)**2)**0.5
  return total_generation_mean,total_generation_average_std

In [45]:
def get_mean_and_std(fsingleoff_varlist):
  fmean_std_off = (np.mean(fsingleoff_varlist))**0.5
  count = 0
  for i in fsingleoff_varlist:
    count += (fmean_std_off-i**0.5)**2
  fstd_meanstd_off = (count/len(fsingleoff_varlist))**0.5
  return fmean_std_off, fstd_meanstd_off

In [46]:
def logg_combi(test_combi):
    file1 = open(combi_log_txt,"a")
    file1.write(str(test_combi))
    file1.write(" - ")
    file1.write(time.strftime("%d.%m.%Y_%H.%M"))
    file1.write("\n")
    file1.close()

def analysis_off_mean(results_overall):
  affe = np.mean(np.moveaxis(results_overall, 1,0),axis = 2)
  np.mean(np.var(affe, axis = 1))**0.5

In [47]:
'''Initialize Variables'''

number_training_categories = 5
NNin1 = 50
NNout1 = number_training_categories

googledrive_path = "/content/drive/MyDrive/Colab Notebooks/"
file_name = "Jax_Quickdraw_Experiment_3"

n_training_epochs = 10 #gradient descent iterations with same data for training the second network
batch_size = 5
n_offsp_epoch = 5 #repeat experiment with same offspring on different category sets


'''keys'''
starting_key = 260 #define starting point
MLP_start_key = 3610 #seed 
numpy_seed = 8450 #in create offsprings

use_pickle = False #load weights
pickle_path = "./pretrained_params/best_weight_0.8463.pkl"
use_best_weights = True

# Main code

In [48]:
def main(var_list):

    std_modifier, n_runs,n_offsp_epoch, n_samples, n_test,number_different_catruns, comment = var_list

    


    
    '''Initialize variables'''
    results_for_every_offspring = []
    father_key = jax.random.PRNGKey(starting_key)
    common_start_acc = 0
    results_array = np.array([])
    collect_stds = []
    collect_means = []
    collect_results = []
    rng_MLP = jax.random.PRNGKey(MLP_start_key)
    MLP_params = init_MLP([NNin1, NNout1], rng_MLP)
    use_pickle = False
    n_offsprings = 1
    temp = 0.05
    result_list = []
    result_list_over_all_runs = []
    avg_std_list = []

    # Defining an optimizer in Jax
    step_size = 1e-3
    opt_init, opt_update, get_params = optimizers.adam(step_size)

    '''Start Logging'''
    save_path =  pathandstuff()
    logg_script(file_name, save_path)
    log_variables()
    logg_GPU()
    logg(comment)

    start_meta = time.time()
    start_overhead = time.time()

    if use_pickle:
      with open(pickle_path, "rb") as input_file:
        father_weights = cPickle.load(input_file)
        print("pickle weights imported")
      offspring_list,randomizer_list = jax_create_offsprings((numpy_seed),n_offsprings, father_weights,std_modifier)

    else:
      father_weights = conv_init(father_key, (batch_size,data_image_size,data_image_size,data_image_depth))
      father_weights = father_weights[1] ## Weights are actually stored in second element of two value tuple
      offspring_list,randomizer_list = jax_create_offsprings((numpy_seed),n_offsprings, father_weights,std_modifier)

    results_list = []
    '''Run multiple runs with the same offspring but different seeds and data'''
    for t in tqdm(range(n_runs)):
      results_overall = []

      for oepoch in range(n_offsp_epoch):
        random_state = (starting_key+oepoch)
        result_list_metaepoch = []

        random_training_categories = random_cat_list(known_list, number_different_catruns, number_training_categories)
        x_train, _, y_train, _ = cat_dataloader(x_train_known_dataset,y_train_known_dataset,random_training_categories, random_state, n_samples, n_test, number_different_catruns)
        _, x_test, _, y_test = cat_dataloader(x_test_known_dataset,y_test_known_dataset,random_training_categories, random_state, n_samples, n_test, number_different_catruns)

        '''Cast to Jax Array'''
        x_train = jnp.array(x_train,dtype = "float32").reshape(number_different_catruns,int((n_samples/batch_size)),batch_size,data_image_size,data_image_size)
        x_test = jnp.array(x_test,dtype = "float32").reshape(number_different_catruns,n_test,data_image_size,data_image_size)
        y_train = jnp.array(y_train).reshape(number_different_catruns,int((n_samples/batch_size)),batch_size)
        y_test = jnp.array(y_test).reshape(number_different_catruns,n_test)

        #print("Shape x_train",np.shape(x_train))
        #print("Shape y_train",np.shape(y_train))
        #print("Shape x_test",np.shape(x_test))
        #print("Shape y_test",np.shape(y_test))

        rng_MLP = jax.random.PRNGKey(random_state)
        MLP_params = init_MLP([NNin1, NNout1], rng_MLP)

        opt_state = opt_init(MLP_params)
        params_with_adam = get_params(opt_state)

        for i in range(len(offspring_list)):
          conv_weights = offspring_list[i]
          result_off,params_trained = jit_vmap_bootstrapp_offspring_MLP(params_with_adam,opt_state,conv_weights,x_train,y_train,x_test,y_test)
          result_list_metaepoch.append(result_off)
        results_overall.append(result_list_metaepoch)

      
      '''Assess Metaepoch'''
      results_list.append(results_overall)
      '''means over all cat combis for every offspring and oepoch'''
      means_of_every_offspring = np.mean(results_overall,axis = 2)
      '''average standard deviation of the mean of every offspring, over all oepochs. How accurate is the fitness evaluation of one offspring?'''
      avg_std = np.mean(np.var(means_of_every_offspring, axis = 0))**0.5
      avg_std_list.append(avg_std)
      #logg(f"\taverage standard deviation of the mean of every offspring: {avg_std:.4f}", )   
      results_for_every_offspring = np.mean(np.mean(results_overall, axis = 0),axis = 1) #stimmt, checked 29.08
      #logg("\tMetaepoch mean: {:.4f}, std: {:.4f}".format(np.mean(results_overall),np.std(results_for_every_offspring))) #stimmt, checked 29.08
      #logg("\tMetaepoch max performer: {:.4f}, min performer: {:.4f}".format(np.max(results_for_every_offspring),np.min(results_for_every_offspring)))
      #logg("\tTime per metaepoch:{:.1f}s\n".format(time.time() - start_meta))

    print()
    print("+++++++++++++++++++")
    print("Assess Run")
    print("+++++++++++++++++++")
    print()

    '''get string list of cat labels'''
    string = ""
    for i in known_npy:
      string+= i+", "
    string
  
    timestamp = time.strftime("%d.%m.%Y_%H.%M")
    ind_string = f"N{n_runs}_T-{n_test}_std{std_modifier}"
    save_path = googledrive_path+costum_name+"/"+timestamp+"_"+ind_string+"/"


    '''Start Logging'''
    pathandstuff(save_path)
    '''Save results after all n_runs'''
    with open(save_path+"results_overall_allruns.pkl", 'wb') as f:
      pickle.dump(result_list_over_all_runs, f, pickle.HIGHEST_PROTOCOL)
      f.close()

    '''deviation of the mean'''
    mean, std = get_gen_mean_std (get_varlist(results_list))

    '''Log to txt'''
    logg(f"average over n_runs: {n_runs}")
    logg(f"average stds of mean over n_runs: {n_runs} -  {mean}")
    logg("number_different_catruns:",number_different_catruns)
    logg("avg_std_list:",str(avg_std_list))
    logg("std_modifier:",std_modifier)
    logg(f"n_samples: {n_samples}")
    logg(f"n_test: {n_test}")
    logg(f"n_oepoch: {n_offsp_epoch}")
    logg(f"average mean per metaepoch: {np.mean(np.mean(result_list_over_all_runs,dtype = object),dtype = object)}")
    logg(f"category labels: {string}")
    logg(f"n_offsprings: {n_offsprings}")
    logg(f"number_different_catruns: {number_different_catruns}")
    logg(f"temp: {temp}")


    '''load df'''
    df_pickle_path = "/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/df_stability_results_v2.pickle"
    with open(df_pickle_path, "rb") as input_file:
          df = cPickle.load(input_file)


    row_location = len(df.index)
    df.loc[row_location,'n_runs'] = n_runs
    df.loc[row_location,'std_modifier'] = std_modifier
    df.loc[row_location,'average stds of mean '] = mean
    df.loc[row_location,'average std of average stds of mean'] = std
    df.loc[row_location,'time stamp'] = timestamp
    df.loc[row_location,'n_samples'] = n_samples
    df.loc[row_location,'n_test'] = n_test
    df.loc[row_location,'average mean per metaepoch'] = np.mean(np.mean(result_list_over_all_runs,dtype = object),dtype = object)
    df.loc[row_location,'category labels'] = string
    df.loc[row_location,'n_offsprings'] = n_offsprings
    df.loc[row_location,'n_offsp_epoch'] = n_offsp_epoch
    df.loc[row_location,'Temperature'] = temp
    df.loc[row_location,'number_different_catruns'] = number_different_catruns
    df.loc[row_location,'time per run'] = (time.time()-start_n_run)/n_runs
    df.loc[row_location,'comment'] = comment
    print(df)
    with open(df_pickle_path, 'wb') as f:
        pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
        f.close()

    df.to_excel(r"/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/stability_results_v2.xlsx", index = False, encoding = "UTF-8")



    


In [49]:

# Defining an optimizer in Jax
step_size = 1e-3
opt_init, opt_update, get_params = optimizers.adam(step_size)

In [50]:
costum_name = "df_direc"

#Initialize pickle dataframe
try:
    os.mkdir("/content/drive/MyDrive/Colab Notebooks/" + costum_name)
except:
  pass

import pandas as pd
df = pd.DataFrame()
df_pickle_path = "/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/df_stability_results_v2.pickle"

with open(df_pickle_path, 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
    f.close()

df.to_excel(r"/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/stability_results_v2.xlsx", index = False, encoding = "UTF-8")

with open("/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/tested_combinations_v2.txt", 'w') as f:
    f.write("")
    f.close()


In [51]:
combi_log_txt = "/content/drive/MyDrive/Colab Notebooks/" + costum_name+ "/tested_combinations_v2.txt"
test_combi_list = []

'''Create all possible combinations of parameter list'''
#n_runs,n_metaepochs,n_offsp_epoch, n_samples, n_test,number_different_catruns, comment = var_list
for xs in itertools.product([0.001], #std_modifier
                            [50],  #n_runs 
                            [10,25,50], #n_offsp_epoch
                            [25,100], #n_samples
                            [75,150,300], #n_test
                            [50,100,200], #number_different_catruns
                            ["check different oepoch and cat for 1 Offspring"]): # comment                     
  test_combi_list.append(xs)

'''test every parameter combination'''
for test_combi in test_combi_list:
  '''load already tested combinations'''
  tested_combis = []
  with open(combi_log_txt, 'r') as fileobj:
    for row in fileobj:
      tested_combis.append(row.replace('\n', '').replace('\t', ''))
  fileobj.close()

  '''check if combi already tested, otherwise run test'''
  if str(test_combi) not in '\t'.join(tested_combis):
    print(test_combi)
    main(test_combi)
    logg_combi(test_combi)


(0.001, 50, 10, 25, 75, 50, 'check differen oepoch and cat for 1 offspring')
on google
Log path: /content/drive/MyDrive/Colab Notebooks/18.10.2022_19.57/
n_training_epochs = 10
n_offsp_epoch = 5
GPU: Tesla T4
check differen oepoch and cat for 1 offspring


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored